In [55]:
#import libraries
import pandas as pd
import urllib.request, json, csv
import requests

In [37]:
#load data from url
url_dataset = "https://raw.githubusercontent.com/onaio/ona-tech/master/data/water_points.json"
with urllib.request.urlopen(url_dataset) as url:
  # load the data to pandas
    df = pd.read_json(url.read())
df.head()

,water_pay,respondent,research_asst_name,water_used_season,_bamboo_dataset_id,_deleted_at,water_point_condition,_xform_id_string,other_point_1km,_attachments,communities_villages,end,animal_number,water_point_id,start,water_connected,water_manager_name,_status,enum_id_1,water_lift_mechanism,districts_divisions,_uuid,grid,date,formhub/uuid,road_available,water_functioning,_submission_time,signal,water_source_type,_geolocation,water_point_image,water_point_geocode,deviceid,locations_wards,water_manager,water_developer,_id,animal_point,water_mechanism_plate,water_lift_mechanism_type,road_type,water_mechanism_plate_units,water_mechanism_plate_no,water_not_functioning,water_source_type_other,simserial,subscriberid
0,no,community,Haruna Mohammed,year_round,,NaT,functioning,_08_Water_points_CV,no,[north_ghana/attachments/1351696546452.jpg],Gumaryili,2012-11-12T11:46:32.454Z,more_500,xxx,2012-10-31T15:11:04.618Z,no,community members,submitted_via_web,5,no,northern,f8bcee72d7a0400fb99ae11bbf804010,grid_further_500_m,2012-10-31,4d41d54d134c4bfa9078571addd819b9,no,yes,2012-11-13 07:13:57,low,dam_dugout,"[10.1892764, -0.66410362]",1351696546452.jpg,10.1892764 -0.66410362 155.10000610351563 5.0,355047040123780,west_mamprusi,community,community,381705,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,no,community,Haruna Mohmmed,year_round,,NaT,functioning,_08_Water_points_CV,yes,[north_ghana/attachments/1351701849971.jpg],Selinvoya,2012-11-12T11:49:36.619Z,50_to_500,xxx,2012-10-31T16:41:49.738Z,no,Amadu Salifu,submitted_via_web,5,yes,northern,c2f6b298955f47ab9f177bee1214141d,grid_further_500_m,2012-10-31,4d41d54d134c4bfa9078571addd819b9,yes,yes,2012-11-13 07:14:04,high,unprotected_well,"[10.28173052, -0.56901122]",1351701849971.jpg,10.28173052 -0.56901122 201.89999389648438 5.0,355047040123780,west_mamprusi,individual,community,381706,yes,no,manual_power,gravel,NaN,NaN,NaN,NaN,NaN,NaN
2,no,community,Haruna Mohammed,year_round,,NaT,functioning,_08_Water_points_CV,yes,[north_ghana/attachments/1351702462336.jpg],Selinvoya,2012-10-31T16:57:37.864Z,50_to_500,xxx,2012-10-31T16:52:02.601Z,no,Sulemana Abdulai,submitted_via_web,5,yes,northern,6bc6d188611d47f6a666cfd1eaa33998,grid_further_500_m,2012-10-31,4d41d54d134c4bfa9078571addd819b9,yes,yes,2012-11-13 07:14:07,high,borehole,"[10.28169238, -0.56962993]",1351702462336.jpg,10.28169238 -0.56962993 202.60000610351563 5.0,355047040123780,west_mamprusi,community,individual,381707,yes,no,manual_power,paved,NaN,NaN,NaN,NaN,NaN,NaN
3,no,community,Haruna Mohammed,year_round,,NaT,functioning,_08_Water_points_CV,yes,[north_ghana/attachments/1351702971561.jpg],Selinvoya,2012-10-31T17:06:55.047Z,50_to_500,xxx,2012-10-31T16:58:46.672Z,no,Haruna Mohammed,submitted_via_web,5,yes,northern,4b28ac4cbba744d79ba4257f772f94d6,grid_further_500_m,2012-10-31,4d41d54d134c4bfa9078571addd819b9,yes,yes,2012-11-13 07:14:14,high,borehole,"[10.28115661, -0.56918339]",1351702971561.jpg,10.28115661 -0.56918339 199.6999969482422 5.0,355047040123780,west_mamprusi,individual,community,381708,yes,no,manual_power,paved,NaN,NaN,NaN,NaN,NaN,NaN
4,no,community,Haruna Mohammed,year_round,,NaT,functioning,_08_Water_points_CV,yes,[north_ghana/attachments/1351703622326.jpg],Selinvoya,2012-10-31T17:15:57.847Z,50_to_500,xxx,2012-10-31T17:08:27.160Z,no,Sulemana,submitted_via_web,5,yes,northern,7893ce5321804f229e533f36e90c9c6f,grid_further_500_m,2012-10-31,4d41d54d134c4bfa9078571addd819b9,yes,yes,2012-11-13 07:14:22,high,borehole,"[10.28044635, -0.56723556]",1351703622326.jpg,10.28044635 -0.56723556 208.6999969482422 5.0,355047040123780,west_mamprusi,community,community,381709,yes,no,manual_power,paved,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# relevant columns
df= df[["communities_villages", "water_point_condition", "water_functioning"]]
df.tail()
#df.size

,communities_villages,water_point_condition,water_functioning
707,Suik,functioning,yes
708,Vundema,broken,yes
709,Vundema,broken,yes
710,Jiniensa,functioning,yes
711,Jagsa,functioning,yes


In [7]:
df['water_point_condition'].value_counts()

functioning           590
broken                 40
abandoned              36
newly_constructed      28
under_construction     16
na_dn                   2
Name: water_point_condition, dtype: int64

In [52]:
class waterPoints:
  #initialize the class attributes
  def __init__(self,data):
    # load data from the url
    self.data = data 
   #The number of water points that are functional
  def functional_water_points (self):
    #filter the data by functional water points
    number_functional=self.data[self.data['water_point_condition'] =="functioning"]

    self.number_functional= number_functional.groupby('water_point_condition').size().reset_index(name="number_functional")
    print("1: The number of water points that are functional:")
    print()
    print(self.number_functional)

  #Calculate number of water points per community
  def community_waterpoints (self):
    self.community_water_points= self.data.groupby("communities_villages").size().reset_index(name='total_water_points')
    print("2: Number of water points per community: ")
    print()
    print(self.community_water_points)

  #The rank for each community by the percentage of broken water points
  def broken_water_points (self):
    #filter by broken points
    bwp = self.data[self.data['water_point_condition'] =="broken"]
    #Convert to dataframe
    self.broken_community_water_points= bwp.groupby("communities_villages").size().reset_index(name='broken_water_points')
    # merge dataframes to get data of broken water points
    self.water_points_merged = pd.merge(self.community_water_points,self.broken_community_water_points, how='outer', on='communities_villages')
    self.water_points_merged['broken_water_points'] = self.water_points_merged['broken_water_points'].fillna(0)

    # # calculate the percentage of broken water points and rank
    self.water_points_merged['broken_water_points_percentage'] = round((self.water_points_merged['broken_water_points'] / self.water_points_merged['total_water_points']) *100, 2)
    self.water_points_merged['rank'] = self.water_points_merged['broken_water_points_percentage'].rank(method='first', ascending=False).astype(int)
    self.water_points_merged.sort_values(by='broken_water_points_percentage', inplace=True, ascending=False)

    print("3: Rank for each community by the percentage of broken water points. ")
    print()
    print(self.water_points_merged)

In [53]:
# instantiate the class
obj=waterPoints(df)
print()
#Calling methods by created object
obj.functional_water_points()
print()
obj.community_waterpoints()
print()
obj.broken_water_points()


1: The number of water points that are functional:

  water_point_condition  number_functional
0           functioning                590

2: Number of water points per community: 

   communities_villages  total_water_points
0              Abanyeri                   4
1           Akpari-yeri                   3
2              Alavanyo                   3
3                 Arigu                  12
4               Badomsa                  27
..                  ...                 ...
60                Zogsa                   6
61                  Zua                  28
62              Zuedema                  18
63              Zukpeni                   6
64               Zundem                  30

[65 rows x 2 columns]

3: Rank for each community by the percentage of broken water points. 

   communities_villages  ...  rank
16             Gbaarigu  ...     1
57              Vundema  ...     2
5                Bandem  ...     3
52                  Soo  ...     4
44            Namgu

# Return data

In [ ]:
# store data in dictionary
def data_analysis():
  data_set = {
      'number_functional': self.community_water_points,
      'number_water_points': self.community_water_points.to_dict('records'),
      'community_ranking': water_points_merged[['communities_villages', 'broken_water_points_percent', 'rank']].to_dict('records')
  }

  # save data to file
  with open("analysis.json", "w", encoding ='utf8') as outfile:
    json.dump(data_set, outfile, indent=4, sort_keys=False)